In [25]:
import tensorflow as tf

import numpy as np
import numpy.linalg as la

import pylab as pl
%pylab inline

import pandas as pd
from pandas import Series, DataFrame

Populating the interactive namespace from numpy and matplotlib


In [5]:
elems = np.array([1, 2, 3, 4, 5, 6])
tf_sum = tf.scan(lambda a, x: a + x, elems)

In [8]:
sess = tf.InteractiveSession()

In [9]:
sess.run(tf_sum)

array([ 1,  3,  6, 10, 15, 21])

# polynomial of matrix evaluated on vector

<tf.Variable 'weights:0' shape=(3,) dtype=float32_ref>

In [12]:
M = tf.Variable(np.diag([2, 3]), name = "M", dtype = np.float32)

In [14]:
v0 = tf.Variable([[2], [3]], name = "v0", dtype = np.float32)

In [15]:
weights = tf.Variable([1, 0, 1], name = "weights", dtype = np.float32)

In [21]:
Mn_v0 = tf.scan(
    fn = lambda vt, w: tf.matmul(M, vt),
    initializer = v0,
    elems = weights,
)

In [29]:
evol1 = tf.matmul(
    tf.stack([weights]),
    tf.reshape(Mn_v0, Mn_v0.shape[:-1])
)

In [18]:
sess.run(tf.global_variables_initializer())

In [30]:
sess.run(Mn_v0)

array([[[  4.],
        [  9.]],

       [[  8.],
        [ 27.]],

       [[ 16.],
        [ 81.]]], dtype=float32)

In [31]:
sess.run(evol1)

array([[ 20.,  90.]], dtype=float32)

In [33]:
Mn_v0_last = tf.foldl(
    fn = lambda v, w: tf.matmul(M, v),
    initializer = v0,
    elems = weights,
)

In [34]:
sess.run(Mn_v0_last)

array([[ 16.],
       [ 81.]], dtype=float32)

In [44]:
evol = tf.scan(
    fn = lambda v, w: (tf.matmul(M, v[0]), v[1] +  w * v[0] ),
    initializer = (v0, tf.zeros_like(v0)),
    elems = weights,
)

In [52]:
sess.run(evol)

(array([[[  4.],
         [  9.]],
 
        [[  8.],
         [ 27.]],
 
        [[ 16.],
         [ 81.]]], dtype=float32), array([[[  2.],
         [  3.]],
 
        [[  2.],
         [  3.]],
 
        [[ 10.],
         [ 30.]]], dtype=float32))

To my great dismay, replacing `scan` with `foldl` does not work:

In [53]:
evol = tf.foldl(
    fn = lambda v, w: (tf.matmul(M, v[0]), v[1] +  w * v[0] ),
    initializer = (v0, tf.zeros_like(v0)),
    elems = weights,
)

ValueError: The two structures don't have the same number of elements. First structure: [<tf.Tensor 'foldl_7/while/Identity:0' shape=() dtype=int32>, <tf.Tensor 'foldl_7/while/Identity_1:0' shape=(2, 2, 1) dtype=float32>], second structure: [<tf.Tensor 'foldl_7/while/add_1:0' shape=() dtype=int32>, [<tf.Tensor 'foldl_7/while/MatMul:0' shape=(2, 1) dtype=float32>, <tf.Tensor 'foldl_7/while/add:0' shape=(2, 1) dtype=float32>]].

In [46]:
power = tf.foldl(
    fn = lambda v, w: tf.matmul(M, v),
    initializer = v0,
    elems = weights,
)

In [47]:
sess.run(power)

array([[ 16.],
       [ 81.]], dtype=float32)

## Cycle through a dimension of length unknown in advance

In [1]:
import tensorflow as tf

In [2]:
a = tf.Variable([1, 2, 3], validate_shape=False)
b = tf.Variable([-1, 1, -1], validate_shape=False)

a.set_shape([None])
b.set_shape([None])

In [3]:
sess = tf.InteractiveSession()

In [4]:
sess.run(tf.global_variables_initializer())

In [5]:
sess.run([a, b])

[array([1, 2, 3]), array([-1,  1, -1])]

In [6]:
cyc = tf.map_fn(
    fn = lambda x: x[0] * x[1], 
    elems = (a, b),
    dtype = tf.int32
)

In [7]:
sess.run(cyc)

array([-1,  2, -3])

In [9]:
sess.run(cyc, {a: [1, 2, 3, 4], b : [-1, 1, -1, 1]})

array([-1,  2, -3,  4])

In [12]:
sess.run(cyc, {a: [1, 2, 3, 4], b : [-1, 1, -1, 1]})

array([-1,  2, -3,  4])

##### It seems however that broadcasting does not work

In [13]:
X = tf.Variable([[1, 2, 3], [4, 5, 6]], validate_shape=False)
X.set_shape([None, 3])

y = tf.Variable([-1, 1, -1])

In [14]:
sess.run(tf.global_variables_initializer())

In [19]:
cyc2 = tf.map_fn(
    fn = lambda x: x[0] * x[1], 
    elems = (X, y[None, :]),
    dtype = tf.int32
)

In [20]:
sess.run(cyc2, {X : [[1, 2, 3]]})

array([[-1,  2, -3]])

In [21]:
sess.run(cyc2)

InvalidArgumentError: TensorArray map_2/TensorArray_1_22: Could not read from TensorArray index 1 because it has not yet been written to.
	 [[Node: map_2/while/TensorArrayReadV3_1 = TensorArrayReadV3[dtype=DT_INT32, _device="/job:localhost/replica:0/task:0/cpu:0"](map_2/while/TensorArrayReadV3_1/Enter, map_2/while/Identity, map_2/while/TensorArrayReadV3_1/Enter_1)]]

Caused by op 'map_2/while/TensorArrayReadV3_1', defined at:
  File "C:\Users\sakalos\moje\python\WinPython\WinPython-64bit-3.5.2.3\python-3.5.2.amd64\lib\runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\sakalos\moje\python\WinPython\WinPython-64bit-3.5.2.3\python-3.5.2.amd64\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\sakalos\moje\python\WinPython\WinPython-64bit-3.5.2.3\python-3.5.2.amd64\lib\site-packages\ipykernel\__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "C:\Users\sakalos\moje\python\WinPython\WinPython-64bit-3.5.2.